In [34]:
import json
import os
from scipy import stats
import pandas as pd
from scipy.stats import normaltest, mannwhitneyu, ttest_ind, shapiro
import numpy as np
from cliffs_delta import cliffs_delta


In [35]:
PROJECT = ["next", "react", "sveltekit"]
TYPE = "TEST"
SCORING_TYPE = "MAX"


In [36]:
all_test_dataset = []
all_non_test_dataset = []
df = pd.DataFrame(columns=["project", "stat", "p_val", "is_diff"])
for project in PROJECT:
    f = open(
        f'../processed_data/{SCORING_TYPE}/{project}/TEST/dim_x.json')
    test_dataset = json.loads(f.read()) 
    all_test_dataset.append(test_dataset) 
    f.close()
    f = open(
        f'../processed_data/{SCORING_TYPE}/{project}/TEST/dim_y.json')
    non_test_dataset = json.loads(f.read())
    all_non_test_dataset.append(non_test_dataset)
    f.close()

    print(f"{project} x: {len(test_dataset)}, y: {len(non_test_dataset)}")
    isnormal = normaltest(test_dataset).pvalue >= 0.005 and normaltest(
        non_test_dataset).pvalue >= 0.005
    
    stat, p_val, r = 0, 0, 0

    if isnormal:
        stat, p_val = ttest_ind(test_dataset, non_test_dataset, alternative='two-sided', trim=0)
        print('ttest')
    else:
        stat, p_val = mannwhitneyu(test_dataset, non_test_dataset, method="auto")
        n1, n2 = len(test_dataset), len(non_test_dataset)
        # r = np.sqrt(stat / (n1 * n2))
        # d = (2 * r) / np.sqrt(1 - r**2)
        # d = 1 - (2*stat)/(n1*n2)
        d = cliffs_delta(test_dataset, non_test_dataset)

        print("Effect size (r):", d)
        print('mannwhitneyu')

    new_line = {"project": project, "stat": stat, "p_val": p_val, "effect_size": d, "is_diff": p_val < 0.05}
    df = pd.concat([df, pd.DataFrame([new_line])], ignore_index=True)

flatten_all_test_dataset = [item for sublist in all_test_dataset for item in sublist]
flatten_all_non_test_dataset = [item for sublist in all_non_test_dataset for item in sublist]
isnormal = normaltest(flatten_all_test_dataset).pvalue >= 0.005 and normaltest(
    flatten_all_non_test_dataset).pvalue >= 0.005

print(f"x: {len(flatten_all_test_dataset)}, y: {len(flatten_all_non_test_dataset)}")
stat, p_val, eff = 0, 0, 0

if isnormal:
    stat, p_val = ttest_ind(flatten_all_test_dataset, flatten_all_non_test_dataset,
                            alternative='two-sided', trim=0)
    print('ttest')
else:
    stat, p_val = mannwhitneyu(flatten_all_test_dataset, flatten_all_non_test_dataset, method="auto")
    n1, n2 = len(flatten_all_test_dataset), len(flatten_all_non_test_dataset)
    # r = np.sqrt(stat / (n1 * n2))
    # d = (2 * r) / np.sqrt(1 - r**2)
    # d = 1 - (2*stat)/(n1*n2)
    d = cliffs_delta(test_dataset, non_test_dataset)

    print("Effect size (r):", d)
    # print('mannwhitneyu', eff)

new_line = {"project": "all", "stat": stat,
            "p_val": p_val, "effect_size": d, "is_diff": p_val < 0.05}
df = pd.concat([df, pd.DataFrame([new_line])], ignore_index=True)
df.to_csv(f"./summary/{SCORING_TYPE}/TEST.csv", index=False)

# with open(f'./summary/{SCORING_TYPE}/raw_test.csv', 'w') as f:
#     f.write('x,\n')
#     for x in flatten_all_test_dataset:
#         f.write(f'{x}\n')

# with open(f'./summary/{SCORING_TYPE}/raw_non_test.csv', 'w') as f:
#     f.write('y,\n')
#     for y in flatten_all_non_test_dataset:
#         f.write(f'{y}\n')


next x: 23918, y: 7962
Effect size (r): (0.21180424517923468, 'small')
mannwhitneyu
react x: 5827, y: 11345
Effect size (r): (-0.180093549405236, 'small')
mannwhitneyu
sveltekit x: 2837, y: 1308
Effect size (r): (-0.10217619076877306, 'negligible')
mannwhitneyu
x: 32582, y: 20615
Effect size (r): (-0.10217619076877306, 'negligible')


/var/folders/j3/30lg3l551hs0j2l0k0pjts680000gn/T/ipykernel_56326/1348280913.py:37: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame([new_line])], ignore_index=True)
/var/folders/j3/30lg3l551hs0j2l0k0pjts680000gn/T/ipykernel_56326/1348280913.py:37: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame([new_line])], ignore_index=True)
/var/folders/j3/30lg3l551hs0j2l0k0pjts680000gn/T/ipykernel_56326/1348280913.py:64: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame([new_line])], ignore_index=True)
